This kernel builds a "study-level" classifier. It's ideally the first approach that many would have tried so far. I have been using this kernel to train study-level classifier and after refining it have decided to share it. I hope you all find it useful. It's written using TensorFlow and uses Weights and Biases for experiment tracking. 

I have also written three other kernels at the start of this competition: 

* [Visualize Bounding Boxes Interactively](https://www.kaggle.com/ayuraj/visualize-bounding-boxes-interactively)

* [[Train] COVID-19 Detection using YOLOv5](https://www.kaggle.com/ayuraj/train-covid-19-detection-using-yolov5)

* [Submission Covid19](https://www.kaggle.com/ayuraj/submission-covid19)

The Submission Covid19 kernel uses the study level model trained using this kernel. 

If you like the work consider showing some love. :)

# 🧰 Imports and Setups

In [ ]:
# Import the latest version of wandb
!pip install -q --upgrade wandb

In [ ]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow_addons as tfa
from tensorflow.keras import mixed_precision


import tensorflow_probability as tfp
tfd = tfp.distributions

import os
import gc
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

# Imports for augmentations. 
from albumentations import (Compose, RandomResizedCrop, Cutout, Rotate, HorizontalFlip, 
                            VerticalFlip, RandomBrightnessContrast, ShiftScaleRotate, 
                            CenterCrop, Resize)

In [ ]:
# W&B related imports
import wandb
print(wandb.__version__)
from wandb.keras import WandbCallback

wandb.login()

In [ ]:
# Increase GPU memory as per the need.
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# 📀 Hyperparameters

In [ ]:
TRAIN_PATH = '../input/siim-covid19-resized-to-224px-png/train/'
AUTOTUNE = tf.data.AUTOTUNE

CONFIG = dict (
    seed = 42,
    num_labels = 4,
    num_folds = 5,
    img_width = 224, # If you change the resolution to 512 reduce batch size. 
    img_height = 224,
    batch_size = 32,
    epochs = 100,
    learning_rate = 1e-3,
    architecture = "CNN",
    competition = 'siim-covid',
    _wandb_kernel = 'ayut',
    infra = "GCP",
)

# 🔨 Build Input Pipeline

In [ ]:
# read training csv file
df = pd.read_csv('../input/siim-covid-merged-train-labels/image_study_total.csv')
df['path'] = df.apply(lambda row: TRAIN_PATH+row.id+'.png', axis=1)

# Group by Study Ids and remove images that are "assumed" to be mislabeled
# Ref: https://www.kaggle.com/c/siim-covid19-detection/discussion/246597
for grp_df in df.groupby('StudyInstanceUID'):
    grp_id, grp_df = grp_df[0], grp_df[1]
    if len(grp_df) == 1:
        pass
    else:
        for i in range(len(grp_df)):
            row = grp_df.loc[grp_df.index.values[i]]
            if row.study_level > 0 and row.boxes is np.nan:
                df = df.drop(grp_df.index.values[i])
                
df = df.drop('boxes', axis=1).reset_index()
Fold = StratifiedKFold(n_splits=CONFIG['num_folds'], shuffle=True, random_state=CONFIG['seed'])
for n, (train_index, val_index) in enumerate(Fold.split(df, df['study_level'])):
    df.loc[val_index, 'fold'] = int(n)
df['fold'] = df['fold'].astype(int)
df.groupby(['fold', 'study_level']).size()

In [ ]:
# Compute Class weights to mitigate class imbalance to some extent. 
fold_samp = df.loc[df.fold!=0]
print(len(fold_samp))

class_weights = compute_class_weight('balanced', 
                                    classes=np.unique(fold_samp['study_level'].values),
                                    y=fold_samp['study_level'].values)

class_weights_dict = {key: val for key, val in zip(np.unique(fold_samp['study_level'].values), class_weights)}
class_weights_dict                                                            

In [ ]:
@tf.function
def decode_image(image):
    # convert the compressed string to a 3D uint8 tensor
    image = tf.image.decode_png(image, channels=3)
    # Normalize image
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image

def load_image(df_dict):
    # Load image
    image = tf.io.read_file(df_dict['path'])
    image = decode_image(image)
    
    # Parse label
    label = df_dict['study_level']
    label = tf.one_hot(indices=label, depth=CONFIG['num_labels'])
    
    return image, label

# Mixup Augmentation policy
@tf.function
def mixup(a, b):
    alpha = [1.0]
    beta = [1.0]

    # unpack (image, label) pairs
    (image1, label1), (image2, label2) = a, b

    # define beta distribution
    dist = tfd.Beta(alpha, beta)
    # sample from this distribution
    l = dist.sample(1)[0][0]

    # mixup augmentation
    img = l*image1+(1-l)*image2
    lab = l*label1+(1-l)*label2

    return img, lab

In [ ]:
CROP_SIZE = CONFIG['img_height']

# Random Resized Crop
transforms = Compose([
            HorizontalFlip(p=0.6),
#             Rotate(limit=6, p=0.3),
            ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=5, p=0.6),
            RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, brightness_by_max=False, p=0.5),
#             Cutout(num_holes=2, max_h_size=int(0.4*CONFIG['img_height']), max_w_size=int(0.4*CONFIG['img_height']), fill_value=0, always_apply=False, p=1.0)
        ])

def aug_fn(image):
    data = {"image":image}
    aug_data = transforms(**data)
    aug_img = aug_data["image"]

    return aug_img.astype(np.float32) 

def augmentations(image, label):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image], Tout=tf.float32)
    aug_img.set_shape((CROP_SIZE, CROP_SIZE, 3))

    return aug_img, label

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

# Simple dataloader
def get_dataloaders(train_df, valid_df):
    trainloader = tf.data.Dataset.from_tensor_slices(dict(train_df))
    validloader = tf.data.Dataset.from_tensor_slices(dict(valid_df))

    trainloader = (
        trainloader
        .shuffle(1024)
        .map(load_image, num_parallel_calls=AUTOTUNE)
        .map(augmentations, num_parallel_calls=AUTOTUNE)
        .batch(CONFIG['batch_size'])
        .prefetch(AUTOTUNE)
    )

    validloader = (
        validloader
        .map(load_image, num_parallel_calls=AUTOTUNE)
        .batch(CONFIG['batch_size'])
        .prefetch(AUTOTUNE)
    )
    
    return trainloader, validloader

# Mixup
def get_mixup_dataloaders(train_df, valid_df):
    trainloader1 = tf.data.Dataset.from_tensor_slices(dict(train_df)).shuffle(1024).map(load_image, num_parallel_calls=AUTOTUNE)
    trainloader2 = tf.data.Dataset.from_tensor_slices(dict(train_df)).shuffle(1024).map(load_image, num_parallel_calls=AUTOTUNE)

    trainloader = tf.data.Dataset.zip((trainloader1, trainloader2))

    # Valid Loader
    validloader = tf.data.Dataset.from_tensor_slices(dict(valid_df))

    trainloader = (
        trainloader
        .shuffle(1024)
        .map(mixup, num_parallel_calls=AUTOTUNE)
        .map(augmentations, num_parallel_calls=AUTOTUNE)
        .batch(CONFIG['batch_size'])
        .prefetch(AUTOTUNE)
    )

    validloader = (
        validloader
        .map(load_image, num_parallel_calls=AUTOTUNE)
        .batch(CONFIG['batch_size'])
        .prefetch(AUTOTUNE)
    )
    
    return trainloader, validloader

In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(20,20))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(np.argmax(label_batch[n].numpy()))
        plt.axis('off')

# Prepare train and valid df
train_df = df.loc[df.fold != 0].reset_index(drop=True)
valid_df = df.loc[df.fold == 0].reset_index(drop=True)

# Prepare dataloader
trainloader, validloader = get_mixup_dataloaders(train_df, valid_df)

# Visualize
image_batches, label_batches = [], []
for _ in range(32//CONFIG['batch_size']):
    image_batch, label_batch = next(iter(trainloader))    
    image_batches.extend(image_batch)
    label_batches.extend(label_batch)
    
show_batch(image_batches, label_batches)

# 🐤 Model

In [ ]:
def get_model():
    base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet')
    base_model.trainabe = True

    inputs = layers.Input((CONFIG['img_height'], CONFIG['img_width'], 3))
    x = base_model(inputs, training=True)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    
    outputs = layers.Dense(CONFIG['num_labels'], kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    outputs = layers.Activation('softmax', dtype='float32', name='predictions')(outputs)
    
    return models.Model(inputs, outputs)

tf.keras.backend.clear_session() 
model = get_model()
model.summary()

In [ ]:
CONFIG['model_name'] = 'effnetb0_mixup'
CONFIG['group'] = 'Effnetb0-Mixup-224'

# 🚄 Train

In [ ]:
# Early stopping regularization
earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=6, verbose=0, mode='min',
    restore_best_weights=True
)

# Reduce learning rate when validation loss gets plateau 
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=CONFIG['learning_rate'])

In [ ]:
# utility to run prediction on out-of-fold validation data. 
def get_predictions(model, validloader, valid_df):
    y_pred = []
    for image_batch, label_batch in tqdm(validloader):
        preds = model.predict(image_batch)
        y_pred.extend(preds)
        
    valid_df['preds'] = y_pred
    
    return valid_df 

# dataframe to collect oof predictions
oof_df = pd.DataFrame()

In [ ]:
# Train the model for 5 folds.
for fold in range(CONFIG['num_folds']):
    print('FOLD: ', fold)
    # Prepare train and valid df
    train_df = df.loc[df.fold != fold].reset_index(drop=True)
    valid_df = df.loc[df.fold == fold].reset_index(drop=True)

    # Prepare dataloaders
    trainloader, validloader = get_mixup_dataloaders(train_df, valid_df)
    
    # Initialize model
    tf.keras.backend.clear_session()
    model = get_model()

    # Compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate=CONFIG['learning_rate'])
    model.compile(optimizer, 
                  loss='categorical_crossentropy', 
                  metrics=['acc', tf.keras.metrics.AUC(curve='ROC')])


    # Update CONFIG dict with the name of the model.
    print('Training configuration: ', CONFIG)

    # Initialize W&B run
    run = wandb.init(project='siim-study-level', 
                     config=CONFIG,
                     group=CONFIG['group'], 
                     job_type='train',
                     tags=['train', 'kfold', 'mixup', 'effnetb0'],
                     notes='training effnetb0 with mixup')
    
    # Train
    _ = model.fit(trainloader, 
                  epochs=CONFIG['epochs'],
                  validation_data=validloader,
                  class_weight=class_weights_dict,
                  callbacks=[WandbCallback(),
                             earlystopper,
                             reduce_lr])
    
    # Evaluate
    loss, acc, auc = model.evaluate(validloader)
    wandb.log({'Val Acc': acc, 'Val AUC-ROC': auc})
    
    # Save model
    model_name = CONFIG['model_name']
    MODEL_PATH = f'models/study_level/{model_name}'
    os.makedirs(MODEL_PATH, exist_ok=True)
    count_models = len(os.listdir(MODEL_PATH))
    
    model.save(f'{MODEL_PATH}/{model_name}_{count_models}.h5')

    # Get Prediction on validation set
    _oof_df = get_predictions(model, validloader, valid_df)
    oof_df = pd.concat([oof_df, _oof_df])

    # Close W&B run
    run.finish()
    
    del model, trainloader, validloader, _oof_df
    _ = gc.collect()
    
os.makedirs('study_level_oof', exist_ok=True)
oof_df.to_csv('study_level_oof/oof_preds.csv', index=False)

The W&B dashboard shown in the GIF below shows the metrics for EfficientNetB0 trained on 512x512 image size on a single V100 GPU. 

The models weights can be found in [here](https://www.kaggle.com/ayuraj/siim-study-level-models).

In the GIF you can:

* See the loss and accuracy metrics. Note that due to Mixup augmentation the validation loss is lower than training loss. This is the case of strong regularization. <br>
* In the table mode of the dashboard, I sorted the VAL ROC-AUC score in descending order. Clearly there is some variance in the score between diffrent folds. 

![img](https://i.imgur.com/kCJNhbp.gif)

# 🦄 CV Score

In [ ]:
oof_df = pd.read_csv('study_level_oof/oof_preds.csv')
oof_df.head()

In [ ]:
def get_argmax(row):
    return [float(val) for val in row.preds.strip('[]').split(' ') if val!='']

oof_df['preds'] = oof_df.apply(lambda row: get_argmax(row), axis=1)

In [ ]:
metric = tf.keras.metrics.CategoricalAccuracy()
metric.update_state(tf.one_hot(oof_df.study_level.values, depth=4), tf.cast(np.array(list(map(np.array, oof_df.preds.values))), tf.float32))
print(f'CV Score: {metric.result().numpy()}')

In [ ]:
metric = tf.keras.metrics.AUC(curve='ROC')
metric.update_state(tf.one_hot(oof_df.study_level.values, depth=4), tf.cast(np.array(list(map(np.array, oof_df.preds.values))), tf.float32))
print(f'CV Score: {metric.result().numpy()}')